In [1]:
%pip install -U opencv-python
%pip install -U matplotlib

  Using cached opencv_python-4.6.0.66-cp36-abi3-win_amd64.whl (35.6 MB)
     ---------------------------------------- 14.6/14.6 MB 240.2 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 7.2/7.2 MB 346.0 kB/s eta 0:00:00
     ---------------------------------------- 163.6/163.6 kB 377.6 kB/s eta 0:00:00
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached fonttools-4.38.0-py3-none-any.whl (965 kB)
     ---------------------------------------- 55.4/55.4 kB 480.4 kB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 443.7 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np;
import cv2 as cv;
import os
from os.path import join as joinPath


In [3]:
from matplotlib import pyplot as plt

## Helper to show image on VSCode. Because cv.imshow doesn't work there.
def showImage(title: str, image: cv.Mat, color: bool = True):
    if (color):
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    plt.imshow(image)
    plt.title(title)
    plt.show()


In [4]:
def otsuThreshold(image: cv.Mat):
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    return cv.threshold(gray, 0, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)[1]

In [5]:
bedsheetImage = cv.imread('samples/Bedsheet_image.jpg')
karcheepImage = cv.imread('samples/Karcheep.jpg')
lightClothImage = cv.imread('samples/Lightweight_Cloth.jpg')
shirtFabricImage = cv.imread('samples/Shirt_fabric.jpg')
towelImage = cv.imread('samples/Towel fabric.jpg')

allImages = [bedsheetImage, karcheepImage, lightClothImage, shirtFabricImage, towelImage]

In [6]:
## Not sure what it does but opencv tutorial does this
## https://docs.opencv.org/4.x/d2/dbd/tutorial_distance_transform.html
def blackBackground(image: cv.Mat) -> cv.Mat:
    src = image.copy()
    src[np.all(src == 255, axis=2)]
    return src


In [7]:
## Does laplacian filtering (without applying above blackBackground)
def laplacianSharpening(image: cv.Mat, coefficient: int = 1) -> cv.Mat:
    kernel = np.array([[1, 1, 1], [1, -8, 1], [1, 1, 1]], dtype=np.float32) * coefficient
    imgLaplacian = cv.filter2D(image, cv.CV_32F, kernel)
    sharp = np.float32(image)
    imgResult = sharp - imgLaplacian
    imgResult = np.clip(imgResult, 0, 255)
    imgResult = imgResult.astype('uint8')
    return imgResult

In [8]:
## Helper to avoid nesting too many parentheses
def pipeline(image: cv.Mat, functions):
    for transform in functions:
        image = transform(image)
    return image

In [9]:
## Distance transform a grayscale image
def distanceTransform(image: cv.Mat, distanceType=cv.DIST_L2, maskSize=3):
    return cv.distanceTransform(image, distanceType, maskSize)

In [10]:
def binaryThreshold(image: cv.Mat) -> cv.Mat:
    return cv.threshold(image, 100, 255, cv.THRESH_BINARY)[1]

In [11]:
def preprocess(image: cv.Mat) -> cv.Mat:
    return pipeline(image, [laplacianSharpening, otsuThreshold, distanceTransform])

In [15]:
image = bedsheetImage
imagePre = preprocess(image)
imagePre8u = imagePre.astype(np.uint8)
contours, _ = cv.findContours(imagePre, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
markers = np.zeroes(imagePre.shape, dtype=np.int32)
for i in range(len(contours)):
    cv.drawContours(markers, contours, i, i+1, -1)
cv.circle(markers, (5,5), 3, (255,255,255), -1)
markers_8u = (markers * 10).astype('uint8')
cv.imshow('Markers', markers_8u)

cv.watershed(image, markers)
